<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ViTLargeConsistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [4]:
%cd content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [5]:
import os
len(os.listdir('Dataset_Consistent/valid/Funny_Consistent/'))

172

In [6]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset_Consistent/")

Resolving data files:   0%|          | 0/1203 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/401 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/402 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-edd842f627080eff/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1203
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 401
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 402
    })
})

In [8]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [9]:
import shutil

In [10]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-large-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [11]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [12]:
prepared_ds = dataset.with_transform(transform)

In [13]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1203
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 401
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 402
    })
})

In [14]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-14-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [15]:
from transformers import ViTForImageClassification

labels = prepared_ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
NUM_CLASSES = 2
BATCH_SIZE = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Some weights of the model checkpoint at google/vit-large-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-large",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.1
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [18]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.345800,0.593409,0.668329
400,0.145400,0.594593,0.708229
600,0.048100,0.655643,0.723192


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1227642664GF
  train_loss               =       0.3028
  train_runtime            =   0:22:56.60
  train_samples_per_second =        3.496
  train_steps_per_second   =        0.439


In [19]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6683
  eval_loss               =     0.5934
  eval_runtime            = 0:00:24.76
  eval_samples_per_second =      16.19
  eval_steps_per_second   =      2.059


In [20]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)

***** eval metrics *****
  test_accuracy           =     0.6841
  test_loss               =     0.5891
  test_runtime            = 0:05:57.36
  test_samples_per_second =      1.125
  test_steps_per_second   =      0.143


In [21]:
predictions_tensor = torch.tensor(metrics.predictions)
predictions_tensor

tensor([[-7.6416e-01,  3.7793e-01],
        [ 6.0303e-01, -9.3652e-01],
        [-1.6445e+00,  1.1533e+00],
        [-8.7354e-01,  5.5908e-01],
        [-1.0553e-01, -4.0918e-01],
        [ 5.7275e-01, -2.4902e-01],
        [-1.2952e-01,  7.1533e-02],
        [ 6.6895e-01, -1.8848e-01],
        [ 4.8877e-01, -2.6782e-01],
        [ 4.5850e-01, -3.6108e-01],
        [ 2.7856e-01, -3.5767e-01],
        [ 2.8784e-01, -1.5125e-03],
        [-1.1572e+00,  1.2080e+00],
        [-9.3018e-01,  1.1445e+00],
        [-7.5378e-02, -1.5747e-01],
        [-6.2207e-01,  1.9507e-01],
        [-9.0479e-01,  3.9502e-01],
        [ 1.2012e-01, -2.9932e-01],
        [ 5.0244e-01, -3.0591e-01],
        [ 1.2048e-01,  3.4229e-01],
        [-3.1055e-01,  6.7383e-01],
        [ 1.9421e-01, -1.9653e-01],
        [-6.4453e-01,  5.0293e-01],
        [ 2.6459e-02, -4.7943e-02],
        [ 3.6475e-01, -2.8711e-01],
        [-6.6064e-01,  4.0479e-01],
        [-5.0842e-02, -3.8403e-01],
        [-5.2441e-01, -5.084

In [22]:
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

torch.Size([402, 2])


(tensor([1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
         0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
         1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 0, 1, 1

In [23]:
preds1 = torch.softmax(predictions_tensor, dim=1)
print(preds1.shape)
preds1 = torch.argmax(preds1, dim=1)
preds1, preds1.shape, torch.sum(preds1==preds, dtype=int)

torch.Size([402, 2])


(tensor([1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
         0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
         1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 0, 1, 1

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [25]:
metrics

PredictionOutput(predictions=array([[-7.64160156e-01,  3.77929688e-01],
       [ 6.03027344e-01, -9.36523438e-01],
       [-1.64453125e+00,  1.15332031e+00],
       [-8.73535156e-01,  5.59082031e-01],
       [-1.05529785e-01, -4.09179688e-01],
       [ 5.72753906e-01, -2.49023438e-01],
       [-1.29516602e-01,  7.15332031e-02],
       [ 6.68945312e-01, -1.88476562e-01],
       [ 4.88769531e-01, -2.67822266e-01],
       [ 4.58496094e-01, -3.61083984e-01],
       [ 2.78564453e-01, -3.57666016e-01],
       [ 2.87841797e-01, -1.51252747e-03],
       [-1.15722656e+00,  1.20800781e+00],
       [-9.30175781e-01,  1.14453125e+00],
       [-7.53784180e-02, -1.57470703e-01],
       [-6.22070312e-01,  1.95068359e-01],
       [-9.04785156e-01,  3.95019531e-01],
       [ 1.20117188e-01, -2.99316406e-01],
       [ 5.02441406e-01, -3.05908203e-01],
       [ 1.20483398e-01,  3.42285156e-01],
       [-3.10546875e-01,  6.73828125e-01],
       [ 1.94213867e-01, -1.96533203e-01],
       [-6.44531250e-01, 

In [27]:
print(accuracy_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

0.6840796019900498


,Test Funny,Test Not Funny
Pred Funny,63,26
Pred Not Funny,101,212


In [28]:
len(os.listdir('/content/content/MyDrive/SummerResearch/Dataset_Consistent/test/Not_Funny_Consistent'))

238

In [29]:
164/402

0.4079601990049751